<a href="https://colab.research.google.com/github/kahramankaya/Anlas-l-rEkonomiYoutube/blob/main/79.video(Repeated%20Measures%20ANOVA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#79.video(Repeated Measures ANOVA)

In [ ]:
import pandas as pd
import numpy as np
#burda size olarak az sayıda 6 tane belırlemısım ama bu test ıcın orneklem sayısının cok fazla olması gerkeır yoksa frıedman uygulanırdı.
data=pd.DataFrame(
{
"kısıler":["A","B","C","D","X","Y"],
"testoncesı":np.random.randint(1000,size=(6)) ,  
"testsonrası1":np.random.randint(1000,size=(6)),
"testsonrası2":np.random.randint(1000,size=(6)),
"testsonrası3":np.random.randint(1000,size=(6)),
}
)
print(data)

  kısıler  testoncesı  testsonrası1  testsonrası2  testsonrası3
0       A         160           362           910           774
1       B         780           493           873           331
2       C         551           132           163            43
3       D         664           534           883           558
4       X         571             7           156            68
5       Y         298           904           838           740


In [ ]:
#pandas melt ile bu verıyı duzgun hale getırelım.
pd =pd.melt(data , id_vars=["kısıler"], value_vars=["testoncesı","testsonrası1","testsonrası2","testsonrası3"])
pd.columns=["kısıler","test","olcum"]
print(pd)

   kısıler          test  olcum
0        A    testoncesı    160
1        B    testoncesı    780
2        C    testoncesı    551
3        D    testoncesı    664
4        X    testoncesı    571
5        Y    testoncesı    298
6        A  testsonrası1    362
7        B  testsonrası1    493
8        C  testsonrası1    132
9        D  testsonrası1    534
10       X  testsonrası1      7
11       Y  testsonrası1    904
12       A  testsonrası2    910
13       B  testsonrası2    873
14       C  testsonrası2    163
15       D  testsonrası2    883
16       X  testsonrası2    156
17       Y  testsonrası2    838
18       A  testsonrası3    774
19       B  testsonrası3    331
20       C  testsonrası3     43
21       D  testsonrası3    558
22       X  testsonrası3     68
23       Y  testsonrası3    740


In [ ]:
#her test ıcın normallık ıcın pingoin kullandık. boylece her grup ıcın normallıge baktı ve hepsının normal dagıldıgını soyledı.
!pip install pingouin
import pingouin as pg
#dv kısmına bagımlı degıskenı atıyoruz.
normallık_testı=pg.normality(data=pd,dv="olcum",group="test",method="shapiro")
print(normallık_testı)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
                     W      pval  normal
test                                    
testoncesı    0.895402  0.347463    True
testsonrası1  0.818764  0.086103    True
testsonrası2  0.974732  0.922580    True
testsonrası3  0.930115  0.580987    True


In [ ]:
#küresellik testi:varyans lar eşit mi değil mi buna bakar. h0 eşit diyor.
#subject kısmı ilk column alınır. within ise hangı gruplar karsılastırmak ıstnıyorsa onun columnu yazılır.
#eger homojenlık saglanmıssa o zaman tekrarlı anova yapısı uygulanabilir.
homojenlik=pg.sphericity(data=pd,dv="olcum",subject="kısıler",within="test")
print(homojenlik)

SpherResults(spher=True, W=0.14680214505937053, chi2=7.141714439042083, dof=5, pval=0.22431069502597029)


In [ ]:
ANOVA=pg.rm_anova(data=pd,dv="olcum",subject="kısıler",within="test")
#p-unc degerıne bak.
#demek ki yapılan testler arasında anlamlı bır farklılık yokmus. eger olsaydı o zaman o zaman bu farkı yaratan grupları bulmak ıcın posthoc testıne gecılır.
print(ANOVA)

  Source  ddof1  ddof2         F     p-unc       ng2       eps
0   test      3     15  0.375334  0.772069  0.063152  0.485982


In [ ]:
#varsayalım ki anova sonucu h0 red edildi. bu farka hangı grupların neden oldugunu bulamk ıcın posthoc testı uygulayacaktık.
#padjust parametresi eklenmesi gerekir dogru sonuc verılebılmesı ıcın. unutma.
posthoc=pg.pairwise_ttests(data=pd,dv="olcum",subject="kısıler",within="test",padjust="bonf")
#p-corr kısmına bak. orası 0.05 ten kucuk ıse demek ki orası farka neden olmus demektır.
print(posthoc)

  Contrast             A             B  Paired  Parametric         T  dof  \
0     test    testoncesı  testsonrası1    True        True  0.311962  5.0   
1     test    testoncesı  testsonrası2    True        True -1.184289  5.0   
2     test    testoncesı  testsonrası3    True        True -0.052404  5.0   
3     test  testsonrası1  testsonrası2    True        True -0.949270  5.0   
4     test  testsonrası1  testsonrası3    True        True -0.793568  5.0   
5     test  testsonrası2  testsonrası3    True        True  0.576734  5.0   

  alternative     p-unc  p-corr p-adjust   BF10    hedges  
0   two-sided  0.767659     1.0     bonf  0.389  0.223229  
1   two-sided  0.289523     1.0     bonf  0.625 -0.427985  
2   two-sided  0.960236     1.0     bonf  0.374 -0.032626  
3   two-sided  0.386075     1.0     bonf  0.528 -0.652794  
4   two-sided  0.463426     1.0     bonf  0.479 -0.251444  
5   two-sided  0.589110     1.0     bonf  0.427  0.380944  


/usr/local/lib/python3.8/dist-packages/pingouin/pairwise.py:28: UserWarning: pairwise_ttests is deprecated, use pairwise_tests instead.
  warnings.warn("pairwise_ttests is deprecated, use pairwise_tests instead.", UserWarning)
